In [2]:
# get dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-01-14 22:52:03--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.2’

input.txt.2         100%[===================>]   1.06M  1.81MB/s    in 0.6s    

2024-01-14 22:52:04 (1.81 MB/s) - ‘input.txt.2’ saved [1115394/1115394]



In [3]:
# load dataset
with open("input.txt", 'r', encoding="utf-8") as f:
    text = f.read()

print(f"Lenght of dataset is {len(text)}")

Lenght of dataset is 1115394


In [6]:
print(text[0:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [15]:
# vocab
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(f"Chars in dataset are {''.join(chars)}")
print(f"vocab_size: {vocab_size}")

Chars in dataset are 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size: 65


In [16]:
# encoder/decoder
c2i = {c:i for i,c in enumerate(chars)}
i2c = {i:c for i,c in enumerate(chars)}

encode = lambda s: [c2i[c] for c in s]
decode = lambda l: "".join(i2c[i] for i in l)

print(encode("hi there!"))
print(decode(encode("hi there!")))

[46, 47, 1, 58, 46, 43, 56, 43, 2]
hi there!


In [17]:
#load the data
import torch
data =  torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[0:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [18]:
# Divide dataset into training and test sets

n = int(0.9 * len(data))
training_data = data[0:n]
val_data = data[n:]

In [19]:
block_size = 8
x = training_data[:block_size]
y = training_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, target is {target}")

When input is tensor([18]), target is 47
When input is tensor([18, 47]), target is 56
When input is tensor([18, 47, 56]), target is 57
When input is tensor([18, 47, 56, 57]), target is 58
When input is tensor([18, 47, 56, 57, 58]), target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [28]:
torch.manual_seed(1337)
block_size = 8 # context lenght for predictions
batch_size = 4 # how many independent sequences will be procesed in parallel

def gen_batch(split):
    data = training_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i+1 : i + block_size + 1] for i in ix])
    return x, y

xb, yb = gen_batch("train")
print("Inputs")
print(xb.shape)
print(xb)
print("Targets")
print(yb.shape)
print(yb)
print("-----------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"When input is {context}, target is {target}")






Inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------
When input is tensor([24]), target is 43
When input is tensor([24, 43]), target is 58
When input is tensor([24, 43, 58]), target is 5
When input is tensor([24, 43, 58,  5]), target is 57
When input is tensor([24, 43, 58,  5, 57]), target is 1
When input is tensor([24, 43, 58,  5, 57,  1]), target is 46
When input is tensor([24, 43, 58,  5, 57,  1, 46]), target is 43
When input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), target is 39
When input is tensor([44]), target is 53
When input is tensor([44, 53]), target is 56
When input is tensor([44, 53, 56]), target is 1
When input is tensor([44,

In [31]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embededing_table = nn.Embedding(vocab_size, vocab_size)  # each column will indicate the probability of the predicted word

    def forward(self, idx, targets):
        logits = self.token_embededing_table(idx) # (B, T, C)
        B, T, C = logits.shape
        logits = logits.view(B*T,C)
        targets = targets.view(B*T)

        loss = F.cross_entropy(logits, targets)
        return logits, loss

m = BigramLanguageModel(vocab_size)
logits, loss =  m(xb, yb)
print(logits.shape)
print(loss.shape)
print(loss)


torch.Size([32, 65])
torch.Size([])
tensor(4.8786, grad_fn=<NllLossBackward0>)
